In [ ]:
import numpy as np
from tqdm import tqdm
import sys
sys.path.append('Knowledge-Distillation/utils/')

# Outputs x_batch, y_batch, and image names
from image_preprocessing_ver1 import ImageDataGenerator
from keras.models import Model

In [ ]:
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
from keras.utils import to_categorical

x_train = x_train.reshape(x_train.shape + (1,))
x_test = x_test.reshape(x_test.shape + (1,))

# y_train = to_categorical(y_train)
# y_test = to_categorical(y_test)

In [ ]:
import os

data_generator = ImageDataGenerator(data_format='channels_last')

batch_size = 1

data_dir = f'{os.getcwd()}\data\mnist'

train_generator = data_generator.flow_from_directory(os.path.join(data_dir, 'train'), target_size=(28, 28), color_mode='grayscale', shuffle=False, batch_size=batch_size)

val_generator = data_generator.flow_from_directory(os.path.join(data_dir, 'test'), target_size=(28, 28), color_mode='grayscale', shuffle=False, batch_size=batch_size)

In [ ]:
# Saves MNIST to dir for use with ImageDataGenerator

# from keras.preprocessing.image import save_img, load_img, img_to_array
# import os
# from collections import defaultdict

# save_path = 'data/mnist/test'
# name_dict = defaultdict(int)
# for i, img in enumerate(x_test):
#     img_array = img_to_array(img)
#     name = os.path.join(save_path, str(y_test[i]), str(y_test[i])+'('+str(name_dict[y_test[i]])+')'+'.jpg')
#     name_dict[y_test[i]] += 1
#     save_img(name, img)

In [ ]:
from keras.models import load_model

model = load_model('mnist_teacher.h5')

# Remove softmax layer
model.pop()

# This model outputs teacher logits
model = Model(model.input, model.layers[-1].output)

In [ ]:
batches = 0
train_logits = {}

for x_batch, _, name_batch in tqdm(train_generator):
    batch_logits = model.predict_on_batch(x_batch)
    
    for index, name in enumerate(name_batch):
        train_logits[name] = batch_logits[index]
        
    batches += 1
    if batches > len(x_train) / batch_size:
        break

In [ ]:
batches = 0
val_logits = {}

for x_batch, _, name_batch in tqdm(val_generator):
    batch_logits = model.predict_on_batch(x_batch)
    
    for index, name in enumerate(name_batch):
        val_logits[name] = batch_logits[index]
        
    batches += 1
    if batches > len(x_test) / batch_size:
        break

In [ ]:
np.save(data_dir + 'mnist_train_logits.npy', train_logits)
np.save(data_dir + 'mnist_val_logits.npy', val_logits)